## Imports

In [2]:
import pandas as pd
import datasets
import numpy as np
import torch
import torch.nn as nn


## Data Preprocessing

In [118]:

file_path = ['rossmann-store-sales/train.csv',
             'rossmann-store-sales/test.csv', 'rossmann-store-sales/store.csv']
store_df = pd.read_csv(file_path[2], low_memory=False)

print(f"Size of the store df: {store_df.shape}")

train_df = pd.read_csv(file_path[0], low_memory=False) # split this into test train for validation
test_df = pd.read_csv(file_path[1], low_memory=False)
train_df = pd.merge(train_df, store_df, how="inner", on="Store")
test_df = pd.merge(test_df, store_df, how="inner", on="Store")

print(f"The column names before dropping are : {train_df.columns.tolist()}")
print(f"Size of the dataset after merging is : {train_df.shape}")

cols_to_drop = ['Open', 'StateHoliday', 'SchoolHoliday', 'CompetitionOpenSinceMonth', 'StoreType', 'Assortment', 'PromoInterval', 'Promo2SinceWeek', 'Promo2SinceYear', 'Promo2', 'CompetitionOpenSinceYear', 'CompetitionDistance', 'Customers']

train_df.drop(cols_to_drop, axis=1, inplace=True)
train_df.fillna(0, inplace=True) # not sure if this is the best way to do it

column_data_types = {
    col: np.int64 if col != 'Date' else 'object' for col in train_df.columns.tolist()} # just making sure everything is int type

train_df = train_df.astype(column_data_types) 

for col in train_df.columns: # fix this, store and state are nominal/not ordered
    train_df[col] = train_df[col].astype('category').cat.as_ordered()

train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['Year'], train_df['Month'], train_df['Day'] = train_df['Date'].dt.year.values, train_df['Date'].dt.month.values, train_df['Date'].dt.day.values
train_df.drop(['Date'], axis=1, inplace=True)

# train_df['State'] have yet to add this column. need to get the store_state csv

train_df.drop_duplicates(inplace=True)

# customers is not categorical hmmm/ okay dropped it/not in Paper

train_data_y = pd.DataFrame(train_df['Sales'])
train_data_x = train_df.drop(['Sales'], axis=1)


# BIL BOI THE BEST


##############################################################################################################
# print(len(df))
# train_data_y.head(10)

# for col in train_data_x.columns:
#     print(f"\nUnique values in {col} column: ")
#     print(train_data_x[col].unique())

# find nan rows
# print(df.dtypes)
# print(train_df.isnull().sum())
# train_df.dropna(inplace=True)  # not sure if should drop/should replace with median or mean imo
# train_df["CompetitionDistance"].replace(
#     np.nan, train_df["CompetitionDistance"].mean(), inplace=True)  # replace with mean
# replace with 0 since for the remaining ones, it means there is no Promo
# print(f"The column names after dropping are : {train_df.columns.tolist()}")

Size of the store df: (1115, 10)
The column names before dropping are : ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']
Size of the dataset after merging is : (1017209, 18)
The column names are : ['Store', 'DayOfWeek', 'Date', 'Sales', 'Promo']


In [119]:
train_data_x.sample(10) # shows random 10 instead of first ten like head

,Store,DayOfWeek,Promo,Year,Month,Day
290947,320,6,0,2013,1,5
659275,725,6,0,2015,5,9
224363,247,4,0,2013,1,31
650380,715,3,1,2014,12,3
340739,374,3,0,2013,12,11
636405,699,3,0,2014,12,10
641968,705,4,1,2014,3,6
1000824,1097,5,0,2013,6,14
54526,60,1,1,2013,6,3
454067,498,3,0,2014,5,28


In [64]:


# Define the neural network class
class EmbeddingNetwork(nn.Module):
    def __init__(self, input_size, embedding_size, output_size):
        super(EmbeddingNetwork, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size) # Embedding layer. The size of the embedding is arbitrary, but the number of rows must match the size of the vocabulary/number of categories. The number of columns is the size of the embedding vectors. 
        self.fc = nn.Linear(embedding_size, output_size) # Fully connected layer or the dense layer. This is where the actual computation happens. The actual NN. We pass the output of the embedding layer to the fully connected layer. The fully connected layer is a linear layer, which means that it performs a linear transformation on the input. The input size is the size of the embedding vectors, and the output size is the size of the output.

    def forward(self, x): # The forward method is called when we pass data through the network.
        embedded = self.embedding(x)
        embedded = embedded.mean(dim=1)  # Average the embeddings
        output = self.fc(embedded)
        return output

# not yet defined. This is just arbitrary

# Create an instance of the neural network
input_size = 100  # Size of the input vocabulary
embedding_size = 50  # Size of the embedding vectors
output_size = 10  # Size of the output
model = EmbeddingNetwork(input_size, embedding_size, output_size)

# Print the model architecture
print(model)


EmbeddingNetwork(
  (embedding): Embedding(100, 50)
  (fc): Linear(in_features=50, out_features=10, bias=True)
)
